<a href="https://www.kaggle.com/code/rich627/math-problem-categorization?scriptVersionId=130166514" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/math-problem-categorization/sample_submission.csv
/kaggle/input/math-problem-categorization/train.csv
/kaggle/input/math-problem-categorization/test.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/math-problem-categorization/train.csv')
print('Category:', len(train_data.category.unique()))

Category: 25


In [3]:
#display limits
pd.set_option('display.max_colwidth', None)
train_data.head()

,problem,category
0,"At a ring toss game at a carnival, Marco throws `8` rings. Some of the rings are worth `5` points, and some are worth `10` points. Marco scores a total of `70` points. How many `5`-point rings and `10`-point rings did Marco toss?",19
1,What is `471 ÷ 100`? Interactive: Students follow steps to create an exploratory area model.,22
2,Follow the directions below to create an equivalent fraction for `1/2` with a denominator of `4`.\n Interactive: Students follow steps to create an exploratory area model.,20
3,Solve the following system by substitution:\n`y=-7x+6`\n`10x+y=0`,14
4,Solve the following system by substitution:\n`-8x+y=0`\n`y=4x+8`,14


In [4]:
print('traing_data:', len(train_data))

traing_data: 250


In [5]:
for i in range(25):
    print(train_data[train_data['category']==i].head(1))

                                                                                                       problem  \
17  Identify the terms, coefficients, variables, and constants in the following expression:\n`4m + 5 + 2n + 3`   

    category  
17         0  
                                                            problem  category
14  Write an expression to match the phrase: "`n` increased by `8`"         1
                                                                                      problem  \
32  Write an expression to match the phrase:\n''`6` is added to the result of `9` times `d`''   

    category  
32         2  
                                                                                                problem  \
38  Write an algebraic expression to match the phrase below.\n"Increase `d` by `7`, then divide by `5`"   

    category  
38         3  
                                                                                   problem  \
57  Write an expressi

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

#download stopwords dict 
nltk.download('stopwords')
nltk.download('punkt')
text = train_data.problem
text.lower = text.apply(lambda x: x.lower())
stop_words = set(stopwords.words('english'))

#split to sigle words
tokens = text.apply(lambda x: word_tokenize(x))

filtered_tokens = tokens.apply(lambda x: [word for word in x if word not in stop_words])
filtered_tokens = filtered_tokens.apply(lambda x: [word for word in x if word not in string.punctuation])
filtered_tokens = filtered_tokens.apply(lambda x: [word for word in x if word.strip() != ''])

#stemming
stemmer = PorterStemmer()
stemmed_tokens = filtered_tokens.apply(lambda x: [stemmer.stem(word) for word in x])
stemmed_tokens[1]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['what',
 '`471',
 '÷',
 '100`',
 'interact',
 'student',
 'follow',
 'step',
 'creat',
 'exploratori',
 'area',
 'model']

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(stemmed_tokens)
X_sequence = tokenizer.texts_to_sequences(stemmed_tokens)
X_paded = pad_sequences(X_sequence)
y = train_data.category
X_paded

array([[  0,   0,   0, ...,  72, 136, 212],
       [  0,   0,   0, ...,  31,  32,  33],
       [  0,   0,   0, ...,  31,  32,  33],
       ...,
       [  0,   0,   0, ..., 107, 910, 911],
       [  0,   0,   0, ...,  24, 371,   3],
       [  0,   0,   0, ...,  11, 612, 174]], dtype=int32)

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_paded, y, test_size=0.8, random_state=0)

In [9]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
import tensorflow as tf
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import LSTM, Dense 
RandomForest = RandomForestClassifier(random_state=0)
AdaBoost = AdaBoostClassifier(random_state=0)
GradientBoosting = GradientBoostingClassifier(random_state=0)
SVM = LinearSVC(random_state=0)
RNN = Sequential()

In [10]:
def accuracy(model):
    model.fit(X_train, y_train)
    pred = model.predict()
    print('Accuarcy', accuracy_socre(pred, y_test))

In [11]:
import matplotlib.pyplot as plt
def plot(model):
    plt.title()
    plt.show()
    

In [12]:
#submissionall = pd.read_csv('/kaggle/input/math-problem-categorization-data/sample_submission.csv')
#submission.category = pred 
#submission.to_csv('submission.csv', index=False)